In [ ]:
import usb.core
import usb.util
import time
import numpy as np
import matplotlib.pyplot as plt
# sudo ls -l /dev/bus/usb/003/005
# sudo chmod 777 /dev/bus/usb/003/005
# ps -ef ,,, all processes
#lsusb

In [ ]:
def get_usb_devices():
    usb_devices = []

    devices = usb.core.find(find_all=True)
    for device in devices:
        vid = hex(device.idVendor)
        pid = hex(device.idProduct)
        usb_devices.append((vid, pid))

    if __name__ == "__main__":
        usb_devices = usb_devices
        for vid, pid in usb_devices:
            print("Vendor ID (VID):", vid)
            print("Product ID (PID):", pid)
            print("----------------------")

    usb.core.find(idVendor=0x0483, idProduct=0x5740)
    device = usb.core.find(idVendor=0x0483, idProduct=0x5740)
    if device is None:
        raise ValueError("Device not found")


def send(data):
    device = usb.core.find(idVendor=0x0483, idProduct=0x5740)
    device.set_configuration(1)


    out_endpoint = None
    in_endpoint = None
    for cfg in device:
        for intf in cfg:
            for ep in intf:
                if usb.util.endpoint_direction(ep.bEndpointAddress) == usb.util.ENDPOINT_OUT:
                    out_endpoint = ep
                elif usb.util.endpoint_direction(ep.bEndpointAddress) == usb.util.ENDPOINT_IN:
                    in_endpoint = ep

    if out_endpoint is None or in_endpoint is None:
        raise ValueError("Both OUT and IN endpoints are required for communication.")

    interface = 1
    # out_endpoint= 0x01
    usb.util.claim_interface(device, interface)
    device.write(out_endpoint.bEndpointAddress, data)
    # response = device.read(in_endpoint.bEndpointAddress, in_endpoint.wMaxPacketSize)
    usb.util.release_interface(device, interface)
    usb.util.dispose_resources(device)
    #print(split_string(response.tobytes().hex(),2))
    #endpoint = device[0][(0, 0)][0]


def HtB(hex_string): #hex_string ex. = "F0 A5 45 76 76 76 76"
    hex_values = hex_string.split()
    binary_values = []
    for hex_value in hex_values:
        decimal_value = int(hex_value, 16)
        binary_value = bin(decimal_value)[2:].zfill(8)
        binary_values.append(binary_value)
    return binary_values


def BtH(binary_string): #binary_string ex = "10001111 00000100 00000011 00000000 00000000 00000000 00001111"
    binary_values = binary_string.split()
    hex_values = []
    for binary_value in binary_values:
        decimal_value = int(binary_value, 2)
        hex_value = hex(decimal_value)[2:].upper().zfill(2)
        hex_values.append(hex_value)
    return hex_values


def list_glue(string_list):
    s=""
    for i in string_list:
        s+=i
    return s


def hex_xor(hex1, hex2):
    int1 = int(hex1, 16)
    int2 = int(hex2, 16)
    xor_result = int1 ^ int2
    hex_result=format(xor_result, '02X')
    return hex_result


def checksum(hex_sequence):
    crc = 0x0000
    hex_values = hex_sequence.split()
    for hex_value in hex_values:
        data = int(hex_value, 16)
        crc ^= data << 8
        for _ in range(8):
            if crc & 0x8000:
                crc = (crc << 1) ^ 0x1021
            else:
                crc <<= 1
    crc &= 0xFFFF
    crcmodem = format(crc, '04X')
    crc0 = format(crc, '04X')
    return "81" + " " + hex_xor(crc0[:2],'55') + " " + "81" + " " + hex_xor(crc0[2:],'55')


def length_cal(command_str):
    l = 10 + len(command_str.split())
    l = hex(l)[2:]
    l = hex_xor(l,'55')
    return "81" + " " + "55" + " "  + "81"+ " "  + l


def message_gen(data_string):
    m = "F0" + " " + length_cal(data_string)+ " " + checksum(data_string) + " " + data_string + " " + "0F"
    return m


def init():
    command= "00 1E 00"
    send(bytes.fromhex(message_gen(command)))


def stop():
    command= "0C 22"
    send(bytes.fromhex(message_gen(command)))


# def get_current_data(): #requests the live data and keep alive signal
#     message = message_gen("08 24 02")
#     return bytes.fromhex(message)

def get_current_data(): #requests the live data and keep alive signal
    message = message_gen("08 25 02")
    return bytes.fromhex(message)

def split_string(string, block_size):
    string = ''.join(string.split())
    num_blocks = len(string) // block_size
    blocks = [string[i:i+block_size] for i in range(0, len(string), block_size)]
    out = ' '.join(blocks)
    return out


def update_general(p1_PW, p1_amp, p2_PW, p2_amp, p3_PW, p3_amp, frequency1,frequency2,channel1,channel2, p4_PW, p4_amp, p5_PW, p5_amp, p6_PW, p6_amp):
    p1= bin(p1_PW)[2:].zfill(12) + bin(int(2*p1_amp)+300)[2:].zfill(10) + "0"*10
    p2= bin(p2_PW)[2:].zfill(12) + bin(int(2*p2_amp)+300)[2:].zfill(10) + "0"*10
    p3= bin(p3_PW)[2:].zfill(12) + bin(int(2*p3_amp)+300)[2:].zfill(10) + "0"*10
    p4= bin(p4_PW)[2:].zfill(12) + bin(int(2*p4_amp)+300)[2:].zfill(10) + "0"*10
    p5= bin(p5_PW)[2:].zfill(12) + bin(int(2*p5_amp)+300)[2:].zfill(10) + "0"*10
    p6= bin(p6_PW)[2:].zfill(12) + bin(int(2*p6_amp)+300)[2:].zfill(10) + "0"*10
    ramp = 3
    block_1_8 = "00010001" #bin(channel1-1)[2:].zfill(4) + bin(channel2-1)[2:].zfill(4)
    block_2_8 = bin(2)[2:].zfill(4) + bin(ramp)[2:].zfill(4)
    block_4_16 = bin(((1000*2)//frequency2))[2:].zfill(15) + "0"
    block_3_16 = bin(((1000*2)//frequency1))[2:].zfill(15) + "0"
    command_block= "04 20" + " " + split_string(list_glue(BtH(split_string(block_1_8 + block_2_8 + block_3_16 + p1 + p2 + p3 + block_2_8 + block_4_16 + p4 +p5 +p6 ,8))),2)
    CB=command_block.split()
    for i in range(len(CB)):
        if CB[i] in ["0F","F0","81","55","0f","f0"]:
            CB[i]="81" + " " + hex_xor(CB[i], '55')
    CB2=''
    for i in CB:
        CB2 = CB2 + i
    command_block=split_string(''.join(CB2.split()), 2)
    ml_message = "F0" +" " + length_cal(command_block) + " " + checksum(command_block) + " " + command_block + " " + "0F"
    return bytes.fromhex(ml_message)


def update_1point(pw, amp, frequency): # sends 2 symmetric points, i.e., one complete wave

    p1= bin(pw)[2:].zfill(12) + bin(int(2*amp)+300)[2:].zfill(10) + "0"*10
    p2= bin(pw)[2:].zfill(12) + bin(int(2*(-amp))+300)[2:].zfill(10) + "0"*10
    ramp = 3
    block_1_8 = "00000001" # Channel - Red 1
    block_2_8 = bin(1)[2:].zfill(4) + bin(ramp)[2:].zfill(4) # number of points = 2
    block_3_16 = bin(((1000*2)//frequency))[2:].zfill(15) + "0" # period value wrt 2x
    command_block= "04 20" + " " + split_string(list_glue(BtH(split_string(block_1_8 + block_2_8 + block_3_16 + p1 + p2, 8))), 2)
    CB=command_block.split()
    for i in range(len(CB)):
        if CB[i] in ["0F","F0","81","55","0f","f0"]:
            CB[i]="81" + " " + hex_xor(CB[i], '55')
    CB2=''
    for i in CB:
        CB2 = CB2 + i
    command_block=split_string(''.join(CB2.split()), 2)
    ml_message = "F0" +" " + length_cal(command_block) + " " + checksum(command_block) + " " + command_block + " " + "0F"
    return bytes.fromhex(ml_message)
    #print(split_string(update_1point(pw=200, amp=10, frequency=50).hex(),2))   # for debugging

def send_message(pw, amp, frequency, timer):
    message1=update_1point(pw, amp, frequency)
    message2=get_current_data()
    init()
    send(message1)
    send(message2)
    for _ in range(timer*10):
        #print(split_string(message1.hex(),2))
        #print(split_string(message2.hex(),2))
        time.sleep(0.1)
    stop()

def incremental_message(pw, imin, imax, frequency, duration, step_size):

    if (imax-imin)%step_size == 0:
        res=0
    else:
        res=1
    message2=get_current_data()
    init()
    for i in range(2*imin,int(2*(imax+res*(step_size-(imax-imin)%(step_size)))+1),int(2*step_size)):
        i=i/2
        if i>imax:
            i=imax
        print("Current Amplitude:", i, "mA")
        message1=update_1point(pw, i, frequency)
        for t in range(int(duration*20)):
            send(message1)
            send(message2)
            time.sleep(0.05)
    #stop()

In [ ]:
send_message(120, 10, 40, 2)

In [ ]:
def list_message(amp_list, pw, frequency, dt): # use multipliers of 0.05 seconds
    message2 = get_current_data()
    update_list, x_values,y_value,l=[],[0],[amp_list[0]],0

    for i in amp_list:
        l+=1
        message1=update_1point(pw, i, frequency)
        update_list.append(message1)
        x_values.append(round(l*dt,3))
        y_value.append(i)
    init()
    if dt<1:
       for j in update_list:
           send(j)
           send(message2)
           time.sleep(dt)
    else:
        for k in update_list:
            for s in range(int(20*dt)):
                send(k)
                send(message2)
                time.sleep(0.05)
    stop()
    plt.step(x_values, y_value,"-")
    plt.xlabel('Time (t)')
    plt.ylabel('Values')
    plt.xlim(0, max(x_values)+dt)
    plt.ylim(0, max(y_value)+1)
    plt.title('Plot of Values vs Time')
    plt.show()

